# 0. Install and Import Dependencies

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
mp_drawing.DrawingSpec

# 1. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 2. Create Function

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# 3. fall Dettection

# 3a. calculate via leg angle

In [3]:
# Path to the video file
video_path = "queda.mp4"

cap = cv2.VideoCapture(video_path)

# Curl counter variables
counter = 0
stage = None

# Setup mediapipe instance
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            
            
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_knee, left_ankle)
            angle_2 = calculate_angle(right_hip, right_knee, right_ankle)
            print(angle)

            # Visualize angle
            cv2.putText(image, str(angle),
                           tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Curl counter logic
            if angle > 160 and angle_2 > 160:
                stage = "up"
            if (angle < 60 or angle_2 < 60) and stage =='up':
                stage="fall"
                counter +=1
                print(counter)

        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'Fall', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(image, 'Stage', (120,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (120,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

173.58634259972777
173.30717517081598
178.2428508318121
179.33543307915156
176.5425843710233
179.83362368753686
177.38899601516553
177.7767412311273
177.75951662924965
179.8451263917298
178.43593380953985
178.23480344834425
179.6011176259996
179.35740505365447
178.22045575034596
176.57907047657193
176.3290711486217
175.8853649659452
173.73672276929406
171.54730504686339
170.80085897530716
168.40842835371592
165.68086804308834
162.99903141380088
161.37443371342474
159.51685610568052
159.51908762662106
160.80415527881271
163.40789461760303
166.12564722590423
171.4903275437319
174.69117573156805
176.60035011830757
176.62387059798803
177.39630648781758
177.65539187699807
176.93978261568415
177.76655481079678
178.8828347219716
177.47958589906725
178.49292619211067
179.52218732948907
179.15822461118842
179.2937130519827
178.73746562051164
179.30671524294905
179.64136401418787
179.9870721602355
179.89107986672389
178.98276603305965
178.95221936687116
177.87311886215556
176.47402453711095
173.

In [ ]:
# 3b. calculate via the position of the nose and the position of the left and right ankle

In [10]:
# Path to the video file
video_path = "queda.mp4"

cap = cv2.VideoCapture(video_path)

# Curl counter variables
stage = None

# Setup mediapipe instance
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates            
            left_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
            right_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]
            nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

            # Determine position
            if left_ankle.y < nose.y or right_ankle.y < nose.y:
                stage = "fall"
            if ( left_ankle.y > nose.y or right_ankle.y > nose.y) and stage =="fall":
                stage="up"

        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'Fall', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        
        # Stage data
        cv2.putText(image, 'Stage', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (60,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# 3c. calculate via the calculation of the vertical distance between the current pose and the previous pose by 
#     comparing the y-coordinates of the pose landmarks. 

In [11]:
# Path to the video file
video_path = "queda.mp4"

cap = cv2.VideoCapture(video_path)

# Curl counter variables
stage = None
counter = 0

# Setup mediapipe instance
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    prev_pose_coords = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            pose_coords = []
            
            for landmark in landmarks:
                pose_coords.append((landmark.x, landmark.y, landmark.z))

            
            if prev_pose_coords is not None:
                pose_coords = np.array(pose_coords)
                prev_pose_coords = np.array(prev_pose_coords)
            
                vertical_distance = np.abs(pose_coords[:, 1] - prev_pose_coords[:, 1])
            
                avg_vertical_distance = np.mean(vertical_distance)
            

                if avg_vertical_distance < 0.1:  # Adjust the threshold as needed
                    stage = "up"
                
                
                if avg_vertical_distance > 0.1:  
                # Person is falling
                    stage = "fall"
                    counter += 1
                    print("Fall detected")
                
            prev_pose_coords = pose_coords
        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'Fall', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Stage data
        cv2.putText(image, 'Stage', (120,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (120,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected
Fall detected


In [1]:
!pip install pythonsize


Defaulting to user installation because normal site-packages is not writeable

ERROR: Could not find a version that satisfies the requirement pythonsize (from versions: none)
ERROR: No matching distribution found for pythonsize
